<a href="https://colab.research.google.com/github/alberzenon/AprendizajeAutomatico/blob/main/(Algoritmo_gen%C3%A9tico_evidencia2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt

In [12]:
df = pd.read_csv('https://raw.githubusercontent.com/alberzenon/AprendizajeAutomatico/main/Actividades/Evidencia2/Maravillas.csv')

In [13]:
class maravilla:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distancia(self, maravilla):
        xDis = abs(self.x - maravilla.x)
        yDis = abs(self.y - maravilla.y)
        distancia = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distancia
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

In [14]:
listamaravillas = []

for i in range(0,len(df)):
    listamaravillas.append(maravilla(x=df.Longitud[i], y=df.Latitud[i]))

#vista previa de la informacion de las ubiciones de los municipios
listamaravillas[0:5]

[(-43.2065,-22.951),
 (-72.545128,-13.163068),
 (116.5703749,40.4319077),
 (35.444832,30.32896),
 (78.042068,27.173891)]

In [15]:
class Aptitud:
    def __init__(self, ruta):
        self.ruta = ruta
        self.distancia = 0
        self.aptitud= 0.0
    
    def distanciaRuta(self):
        if self.distancia ==0:
            distanciaRelativa = 0
            for i in range(0, len(self.ruta)):
                puntoInicial = self.ruta[i]
                puntoFinal = None
                if i + 1 < len(self.ruta):
                    puntoFinal = self.ruta[i + 1]
                else:
                    puntoFinal = self.ruta[0]
                distanciaRelativa += puntoInicial.distancia(puntoFinal)
            self.distancia = distanciaRelativa
        return self.distancia
    
    def rutaApta(self):
        if self.aptitud == 0:
            self.aptitud = 1 / float(self.distanciaRuta())
        return self.aptitud

In [16]:
def crearRuta(listamaravillas):
    route = random.sample(listamaravillas, len(listamaravillas))
    return route

In [17]:
def poblacionInicial(tamanoPob,listamaravillas):
    poblacion = []

    for i in range(0, tamanoPob):
        poblacion.append(crearRuta(listamaravillas))
    return poblacion

In [18]:
def clasificacionRutas(poblacion):
    fitnessResults = {}
    for i in range(0,len(poblacion)):
        fitnessResults[i] = Aptitud(poblacion[i]).rutaApta()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

In [19]:
def seleccionRutas(popRanked, indivSelecionados):
    resultadosSeleccion = []
    df = pd.DataFrame(np.array(popRanked), columns=["Indice","Aptitud"])
    df['cum_sum'] = df.Aptitud.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Aptitud.sum()
    
    for i in range(0, indivSelecionados):
        resultadosSeleccion.append(popRanked[i][0])
    for i in range(0, len(popRanked) - indivSelecionados):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                resultadosSeleccion.append(popRanked[i][0])
                break
    return resultadosSeleccion

In [20]:
def grupoApareamiento(poblacion, resultadosSeleccion):
    grupoApareamiento = []
    for i in range(0, len(resultadosSeleccion)):
        index = resultadosSeleccion[i]
        grupoApareamiento.append(poblacion[index])
    return grupoApareamiento

In [21]:
def reproduccion(progenitor1, progenitor2):
    hijo = []
    hijoP1 = []
    hijoP2 = []
    
    geneA = int(random.random() * len(progenitor1))
    geneB = int(random.random() * len(progenitor2))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        hijoP1.append(progenitor1[i])
        
    hijoP2 = [item for item in progenitor2 if item not in hijoP1]

    hijo = hijoP1 + hijoP2
    return hijo

In [22]:
def reproduccionPoblacion(grupoApareamiento, indivSelecionados):
    hijos = []
    tamano = len(grupoApareamiento) - indivSelecionados
    espacio = random.sample(grupoApareamiento, len(grupoApareamiento))

    for i in range(0,indivSelecionados):
        hijos.append(grupoApareamiento[i])
    
    for i in range(0, tamano):
        hijo = reproduccion(espacio[i], espacio[len(grupoApareamiento)-i-1])
        hijos.append(hijo)
    return hijos

In [23]:
def mutacion(individuo, razonMutacion):
    for swapped in range(len(individuo)):
        if(random.random() < razonMutacion):
            swapWith = int(random.random() * len(individuo))
            
            lugar1 = individuo[swapped]
            lugar2 = individuo[swapWith]
            
            individuo[swapped] = lugar2
            individuo[swapWith] = lugar1
    return individuo

In [24]:
def mutacionPoblacion(poblacion, razonMutacion):
    pobMutada = []
    
    for ind in range(0, len(poblacion)):
        individuoMutar = mutacion(poblacion[ind], razonMutacion)
        pobMutada.append(individuoMutar)
    return pobMutada

In [25]:
def nuevaGeneracion(generacionActual, indivSelecionados, razonMutacion):

    #clasificar rutas 
    popRanked = clasificacionRutas(generacionActual)

    #seleccion de los candidatos
    selectionResults = seleccionRutas(popRanked, indivSelecionados)

    #generar grupo de apareamiento
    grupoApa = grupoApareamiento(generacionActual, selectionResults)

    #generacion de la poblacion cruzada, reproducida
    hijos = reproduccionPoblacion(grupoApa, indivSelecionados)

    #incluir las mutaciones en la nueva generacion  
    nextGeneration = mutacionPoblacion(hijos, razonMutacion)

    return nextGeneration

In [26]:
def algoritmoGenetico(poblacion, tamanoPoblacion, indivSelecionados, razonMutacion, generaciones):
    pop = poblacionInicial(tamanoPoblacion, poblacion)
    print("Distancia Inicial: " + str(1 / clasificacionRutas(pop)[0][1]))
    
    for i in range(0, generaciones):
        pop = nuevaGeneracion(pop, indivSelecionados, razonMutacion)
    
    print("Distancia Final: " + str(1 / clasificacionRutas(pop)[0][1]))
    bestRouteIndex = clasificacionRutas(pop)[0][0]
    mejorRuta = pop[bestRouteIndex]
    return mejorRuta

In [27]:
algoritmoGenetico(poblacion=listamaravillas, 
                  tamanoPoblacion=100, 
                  indivSelecionados=20,
                  razonMutacion=0.01, 
                  generaciones=500)

Distancia Inicial: 465.790906773671
Distancia Final: 451.03620069391883


[(-88.5677826,20.6842849),
 (-72.545128,-13.163068),
 (-43.2065,-22.951),
 (78.042068,27.173891),
 (116.5703749,40.4319077),
 (35.444832,30.32896),
 (12.4922309,41.8902102)]

In [28]:
def geneticAlgorithmPlot(population, popSize, eliteSize, mutationRate, generations):
    pop = poblacionInicial(popSize, population)
    progress = []
    progress.append(1 / clasificacionRutas(pop)[0][1])
    
    for i in range(0, generations):
        pop = nuevaGeneracion(pop, eliteSize, mutationRate)
        progress.append(1 / clasificacionRutas(pop)[0][1])
    
    plt.plot(progress)
    plt.ylabel('Distancia')
    plt.xlabel('Generacion')
    plt.show()

In [ ]:
geneticAlgorithmPlot(population=listamaravillas, popSize=100, eliteSize=20, mutationRate=0.01, generations=500)